In [9]:
import pandas as pd
import sqlalchemy as sa
from src.Utils.HelperFunction.helper import fetchDataMssql,encrypt_data,generate_deterministic_key,generate_non_deterministic_key
from src.Utils.connectors.connector import connectMssql
import json
import re ,ast
from src.Utils.anonymiser.anonymiser import anonymize
import numpy as np

import datetime

fake = Faker()

In [2]:
import os

In [3]:
os.curdir

'.'

In [4]:
with open(r"\Users\dhanush.shetty\DContracts_DQP\src\config\config.json","r") as j:
    conf_=json.load(j)

In [ ]:
access_= pd.read_csv(r"C:\Users\dhanush.shetty\DContracts_DQP\src\access_control\Access_table.csv")
  

In [8]:
meta_data = pd.read_csv(r"C:\Users\dhanush.shetty\DContracts_DQP\src\metadata\Metadata_mssql_adventureworks.csv")

In [6]:
classified_df=pd.read_csv(r"C:\Users\dhanush.shetty\presidio_demo\data\Classified_metadata.csv")

In [ ]:
for user_ in conf_:
    SERVER_=conf_[user_].get("SERVER_NAME")
    DATABASE_=conf_[user_].get("DATABASE_NAME")
    SCHEMA_=conf_[user_].get("SCHEMA_NAME")
    PASSWORD_=conf_[user_].get("PASSWORD")
    #get tables
    for table_ in conf_[user_].get("TABLE_REQUIRED"):
        if user_ in str(access_["USER_ACCESS"][access_["Table Name"]==table_].values):
            #fetch data into df
            # data_ = fetchDataMssql(conn_,table_,"*")
            #columns 
            # columns_name=list(classified_df[classified_df["Table Name"]==table_]["Column Name"])
            # data_df =  pd.DataFrame.from_records(data_,columns=[columns_name])
            
            # data_df.to_csv(f"C:\\Users\\dhanush.shetty\\presidio_demo\\data\\data_{table_}.csv",index=False)
            # data_df = pd.read_csv(f"C:\\Users\\dhanush.shetty\\presidio_demo\\data\\data_{table_}.csv")
            columns_to_retain = conf_[user_]["Columns_to_retain"]
            columns_to_discard = conf_[user_]["Columns_to_discard"]
            columns_to_custom_anonymise = list(conf_[user_]["Columns_for_custom_anonymise"].keys())
            #iterate column and tag from classified data
            for column_,tag_ in classified_df[["Column Name","Tag"]][((classified_df["Table Name"]==table_)\
                                                                     & \
                                                                       ((classified_df["Pii"]=="Yes") \
                                                                        | (classified_df["Column Name"].isin(columns_to_retain) \
                                                                        | (classified_df["Column Name"].isin(columns_to_custom_anonymise) )))\
                                                                     & \
                                                                        (~classified_df["Column Name"].isin(columns_to_discard)))\
                                                                    ].values:
                
                    #seperate rules from config 
                    if column_ not in  columns_to_custom_anonymise :
                        #skip column with no tags
                        if str(tag_) != "nan":
                            rule_ = conf_[user_].get(f"{tag_}")
                            #debug for checking values
                            # print(user_,column_,tag_,rule_)
                            #check if rules if encryption is asked to register keys and user into log for security purpose
                            if [x for x in rule_.keys()][0] == "encrypt":
                                # check if its deterministic or non deterministic type of encryption as both requires different keys
                                #debug
                                # print("yes")
                                if rule_["encrypt"] == "Deterministic":
                                    key_ = generate_deterministic_key(f"{user_}_{datetime.datetime.now()}")
                                    rule_["encrypt"]=f"{key_}:::Deterministic"
                                elif rule_["encrypt"] == "Non-Deterministic":
                                    key_ = generate_non_deterministic_key()
                                    # print(type(key_))
                                    rule_["encrypt"]=f"{key_}:::Non-Deterministic"
                                # print(rule_)
                                #write a log for encryption 
                                pd.DataFrame([[user_,DATABASE_,table_,column_,rule_["encrypt"].split(":::")[0],rule_["encrypt"].split(":::")[1],datetime.datetime.now()]],columns=["user","database","table","column","key","type","time"]).to_csv("Encrypt_log.csv",mode="a",index=False)
                            
                            #apply given rules from config to data
                            data_df[column_] = data_df[column_].apply(lambda x : anonymize(str(x),rule_,tag_))
                        else:
                            continue
                    else:
                        rule_ = conf_[user_]["Columns_for_custom_anonymise"][column_]
                        #debug for checking values
                        # print(user_,column_,tag_,rule_)
                        #check if rules if encryption is asked to register keys and user into log for security purpose
                        if [x for x in rule_.keys()][0] == "encrypt":
                            # check if its deterministic or non deterministic type of encryption as both requires different keys
                            #debug
                            # print("yes")
                            if rule_["encrypt"] == "Deterministic":
                                key_ = generate_deterministic_key(f"{user_}_{datetime.datetime.now()}")
                                rule_["encrypt"]=f"{key_}:::Deterministic"
                            elif rule_["encrypt"] == "Non-Deterministic":
                                key_ = generate_non_deterministic_key()
                                # print(type(key_))
                                rule_["encrypt"]=f"{key_}:::Non-Deterministic"
                            # print(rule_)
                            #write a log for encryption 
                            key,type_ = rule_["encrypt"].split(":::")
                            pd.DataFrame([[user_,DATABASE_,table_,column_,key,type_,datetime.datetime.now()]],columns=["user","database","table","column","key","type","time"]).to_csv("Encrypt_log.csv",mode="a",index=False)
                        
                        #apply given rules from config to data
                        data_df[column_] = data_df[column_].apply(lambda x : anonymize(str(x),rule_,tag_))
                  
            #write data into csv      
            data_df.to_csv(f"{user_}_{table_}_19.csv",index=False)
            print(f"table {table_} created")  
        else:
            print(f"user {user_} has no access table {table_}")
        

sqlalchemy instance Created
Constructed query: SELECT * FROM DIMPRODUCT;
sqlalchemy instance
table DIMPRODUCT created
sqlalchemy instance Created
Constructed query: SELECT * FROM FACTRESELLERSALES;
sqlalchemy instance
table FACTRESELLERSALES created
sqlalchemy instance Created
Constructed query: SELECT * FROM DIMSALESTERRITORY;
sqlalchemy instance
table DIMSALESTERRITORY created
sqlalchemy instance Created
Constructed query: SELECT * FROM DIMCUSTOMER;
sqlalchemy instance
yes
{'encrypt': 'b\'6\\x90\\xc7\\xf3"\\x83\\x89\\x94\\xdd\\xef\\x0e?P\\xc7\\x1eU\':::Deterministic'}
yes
<class 'str'>
{'encrypt': 'K3umf87dWRbyODuHXkNc49pYiZ3PPEn9ksBu3laDJjs=:::Non-Deterministic'}
yes
{'encrypt': 'K3umf87dWRbyODuHXkNc49pYiZ3PPEn9ksBu3laDJjs=:::Non-Deterministic'}
table DIMCUSTOMER created
sqlalchemy instance Created
Constructed query: SELECT * FROM FACTRESELLERSALES;
sqlalchemy instance
table FACTRESELLERSALES created
